In [65]:
from random import randint, choice

shops = ['Novus', 'ATB', 'Varus', 'Silpo', 'Megamarket']
products = {
    'bread' : 20,
    'milk' : 30,
    'butter' : 50,
    'apple' : 15,
    'sausage' : 150,
    'chiken' : 120,
    'tea' : 55,
    'banana' : 45,
    'water' : 15,
    'fish' : 80,
}
bills_count = 100000
items_count = bills_count * 5

bills = []
for id in range(0, bills_count):
    shop = choice(shops)
    month = randint(1, 12)
    day = randint(1, 28)
    date = f'{randint(2017, 2022)}-{month if month>9 else "0"+str(month)}-{day if day>9 else "0"+str(day)}'
    bills.append([id, shop, date])

plist = list(products.keys())
items = []
for i in range(0, items_count):
    pname = choice(plist)
    count = randint(1, 6)
    price = products[pname] + randint(-5, 5)
    items.append([randint(0, bills_count-1), pname, count, price])

In [66]:
import sqlite3 as sql

con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS `bill`")
    cur.execute("CREATE TABLE IF NOT EXISTS `bill` (`id` INTEGER, `name` STRING, `date` STRING)")
    for bill in bills:
        cur.execute(f"INSERT INTO `bill` VALUES ('{bill[0]}', '{bill[1]}', '{bill[2]}')")
    con.commit()
    
    cur.execute("DROP TABLE IF EXISTS `item`")
    cur.execute("CREATE TABLE IF NOT EXISTS `item` (`bill_id` INTEGER, `name` STRING, `count` INTEGER, `price` INTEGER)")
    for item in items:
        cur.execute(f"INSERT INTO `item` VALUES ('{item[0]}', '{item[1]}', '{item[2]}', '{item[3]}')")
    
    con.commit()
    cur.close()

In [68]:
import sqlite3 as sql

con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("SELECT * FROM `item` LIMIT 10")
    rows = cur.fetchall()
    for row in rows:
        print(row)

    con.commit()
    cur.close()

(75606, 'chiken', 5, 118)
(80438, 'milk', 2, 33)
(22153, 'fish', 4, 79)
(7063, 'sausage', 3, 150)
(83154, 'milk', 3, 30)
(96255, 'bread', 2, 18)
(33404, 'bread', 3, 25)
(47944, 'bread', 5, 15)
(11193, 'sausage', 6, 145)
(74687, 'apple', 1, 16)


In [69]:
#Порахувати кількість проданового товару
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("SELECT sum(`count`) FROM `item`")
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[(1750532,)]


In [57]:
#Порахувати вартість проданого товару
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("SELECT sum(`count`*`price`) FROM `item`")
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[(1024725,)]


In [70]:
#Порахувати вартість проданого товару за період
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("SELECT bill.name, sum(`count`*`price`) FROM `item` JOIN `bill` ON `id`=`bill_id` WHERE `date` BETWEEN '2019-01-01' AND '2020-12-31' GROUP BY bill.name")
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[('ATB', 6721705), ('Megamarket', 6899945), ('Novus', 6761530), ('Silpo', 6808531), ('Varus', 6817003)]


In [71]:
#Порахувати скільки було придбано товару А в мазазині В за період С
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("SELECT sum(`count`) FROM `item` JOIN `bill` ON `id`=`bill_id` WHERE item.name='milk' AND bill.name='Novus' AND `date` BETWEEN '2019-01-01' AND '2020-12-31'")
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[(12011,)]


In [72]:
#Порахувати скільки було придбано товару А в усіх магазинах за період С
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("SELECT sum(`count`) FROM `item` JOIN `bill` ON `id`=`bill_id` WHERE item.name='milk' AND `date` BETWEEN '2019-01-01' AND '2020-12-31'")
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[(58403,)]


In [73]:
#Порахувати сумарну виручку магазинів за період С
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("SELECT sum(`count`*`price`) FROM `item` JOIN `bill` ON `id`=`bill_id` WHERE `date` BETWEEN '2019-01-01' AND '2020-12-31'")
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[(34008714,)]


In [77]:
#Вивести топ 10 купівель товарів по два за період С (наприклад масло, хліб - 1000 разів)
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("""
    SELECT a.name, b.name, count(*) cnt
    FROM `item` a JOIN `item` b ON a.bill_id=b.bill_id AND a.name<>b.name
    JOIN `bill` ON `id`=a.bill_id 
    WHERE `date` BETWEEN '2019-01-01' AND '2020-12-31'
    AND a.name <> b.name
    GROUP BY a.name, b.name
    ORDER BY cnt desc
    LIMIT 10
    """)
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[('bread', 'tea', 8648), ('tea', 'bread', 8648), ('bread', 'chiken', 8618), ('chiken', 'bread', 8618), ('sausage', 'water', 8536), ('water', 'sausage', 8536), ('butter', 'milk', 8493), ('milk', 'butter', 8493), ('butter', 'sausage', 8486), ('sausage', 'butter', 8486)]


In [75]:
#Вивести топ 10 купівель товарів по три за період С (наприклад молоко, масло, хліб - 1000 разів)
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("""
    SELECT a.name, b.name, c.name, count(*) cnt
    FROM `item` a JOIN `item` b ON a.bill_id=b.bill_id
    JOIN `item` c ON a.bill_id=c.bill_id
    JOIN `bill` ON `id`=a.bill_id 
    WHERE `date` BETWEEN '2019-01-01' AND '2020-12-31'
    AND a.name <> b.name
    AND a.name <> c.name
    AND b.name <> c.name
    GROUP BY a.name, b.name, c.name
    ORDER BY cnt desc
    LIMIT 10
    """)
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[('bread', 'chiken', 'tea', 4435), ('bread', 'tea', 'chiken', 4435), ('chiken', 'bread', 'tea', 4435), ('chiken', 'tea', 'bread', 4435), ('tea', 'bread', 'chiken', 4435), ('tea', 'chiken', 'bread', 4435), ('banana', 'bread', 'tea', 4380), ('banana', 'tea', 'bread', 4380), ('bread', 'banana', 'tea', 4380), ('bread', 'tea', 'banana', 4380)]


In [76]:
#Вивести топ 10 купівель товарів по чотири за період С
con = sql.connect('market.db')
with con:
    cur = con.cursor()
    cur.execute("""
    SELECT a.name, b.name, c.name, d.name, count(*) cnt
    FROM `item` a JOIN `item` b ON a.bill_id=b.bill_id
    JOIN `item` c ON a.bill_id=c.bill_id
    JOIN `item` d ON a.bill_id=d.bill_id
    JOIN `bill` ON `id`=a.bill_id 
    WHERE `date` BETWEEN '2019-01-01' AND '2020-12-31'
    AND a.name <> b.name
    AND a.name <> c.name
    AND a.name <> d.name
    AND b.name <> c.name
    AND b.name <> d.name
    AND c.name <> d.name
    GROUP BY a.name, b.name, c.name, d.name
    ORDER BY cnt desc
    LIMIT 10
    """)
    answ = cur.fetchall()
    print(answ)

    con.commit()
    cur.close()

[('apple', 'banana', 'milk', 'water', 2294), ('apple', 'banana', 'water', 'milk', 2294), ('apple', 'milk', 'banana', 'water', 2294), ('apple', 'milk', 'water', 'banana', 2294), ('apple', 'water', 'banana', 'milk', 2294), ('apple', 'water', 'milk', 'banana', 2294), ('banana', 'apple', 'milk', 'water', 2294), ('banana', 'apple', 'water', 'milk', 2294), ('banana', 'milk', 'apple', 'water', 2294), ('banana', 'milk', 'water', 'apple', 2294)]
